<a href="https://colab.research.google.com/github/dvschultz/stylegan3/blob/main/SG3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# StyleGAN3

By [Derrick Schultz](https://twitter.com/dvsch), with contributions from [crimeacs](https://twitter.com/EarthML1)

Just starting this...expect more updates soon.

If you find this helpful, please consider backing me on [Patreon](https://www.patreon.com/bustbright) or becoming a [YouTube channel member](https://www.youtube.com/channel/UCaZuPdmZ380SFUMKHVsv_AA/join).

## Setup

In [1]:
!nvidia-smi -L

GPU 0: NVIDIA GeForce RTX 2080 Ti (UUID: GPU-ff73e77a-b8d4-004e-34cf-4d49ed29e5df)


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import os
if os.path.isdir("/content/drive/MyDrive/colab-sg3"):
    %cd "/content/drive/MyDrive/colab-sg3/stylegan3/"
elif os.path.isdir("/content/drive/"):
    #install script
    %cd "/content/drive/MyDrive/"
    !mkdir colab-sg3
    %cd colab-sg3
    !git clone https://github.com/dvschultz/stylegan3
    %cd stylegan3
    !mkdir downloads
    !mkdir datasets
    !mkdir pretrained
    !gdown --id 1-5xZkD8ajXw1DdopTkH_rAoCsD72LhKU -O /content/drive/MyDrive/colab-sg2-ada-pytorch/stylegan2-ada-pytorch/pretrained/wikiart.pkl
else:
    !git clone https://github.com/dvschultz/stylegan3
    %cd stylegan3
    !mkdir downloads
    !mkdir datasets
    !mkdir pretrained
    %cd pretrained
    !gdown --id 1-5xZkD8ajXw1DdopTkH_rAoCsD72LhKU
    %cd ../

!pip install Ninja opensimplex

This cell will update to the latest repo. Git and Drive/Colab don’t play as nicely as I’d like so 🤞. The other option is to delete your folder in Drive (after saving out `/results` and `/datasets`!) and running the script above to replace the entire folder.

In [ ]:
%cd "/content/drive/My Drive/colab-sg2-ada-pytorch/stylegan2-ada-pytorch"
!git config --global user.name "test"
!git config --global user.email "test@test.com"
!git fetch origin
!git pull
!git stash
!git checkout origin/main -- train.py gen_images.py gen_video.py README.md training/training_loop.py

## Convert/Create Dataset
Pass a folder of images (just .pngs? TK) to create a zip file.

In [ ]:
!python dataset_tool.py --source=/content/tmp/drawn-gems-1024 --dest=./datasets/drawn-gems-1024.zip

## Training

Before you start training, read [this](https://github.com/dvschultz/stylegan3/blob/main/docs/configs.md).

Working Notes:
- It appears that you must use an SG3 pre-trained model for transfer learning. I _think_ you also want to match config to the pretrained model (`t` with `t`, `r` with `r`).
- For an `A100` I’ve found you can use a `--batch-gpu=8`. For other GPUs I recommend `--batch-gpu=4`.
- I see `~205 sec/kimg` on A100s, and `~325 sec/kimg` on V100s (1024, `r` config). This seems slightly slower than what [NVIDIA reports.](https://github.com/dvschultz/stylegan3/blob/main/docs/configs.md)

In [ ]:
!python train.py --help

In [ ]:
!python train.py --outdir=./results --cfg=stylegan3-r --data=./datasets/drawn-gems-1024.zip \
--gpus=1 --batch=32 --batch-gpu=4 --gamma=10.0 --mirror=1 --kimg=5000 --snap=1 \
--resume=/content/drive/MyDrive/colab-sg3/stylegan3/results/00014-stylegan3-r-drawn-gems-1024-gpus1-batch32-gamma10/network-snapshot-000104.pkl --metrics=None

## Image Generation

In [ ]:
!python gen_images.py --help

In [ ]:
!python gen_images.py --outdir=out --trunc=1 --seeds=2 \
    --network=https://api.ngc.nvidia.com/v2/models/nvidia/research/stylegan3/versions/1/files/stylegan3-r-afhqv2-512x512.pkl

## Video Generation


In [ ]:
!python gen_video.py --help

In [ ]:
!python gen_video.py --output=/content/lerp.mp4 --trunc=1 --seeds=100-124 --grid=1x1 --w-frames=72 \
    --network=/content/drive/MyDrive/colab-sg3/stylegan3/results/00014-stylegan3-r-drawn-gems-1024-gpus1-batch32-gamma10/network-snapshot-000104.pkl